In [1]:
#imports
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
from torch import nn
import pickle
from torch.utils.tensorboard import SummaryWriter
import os
import shutil
import numpy as np
#project specific imports
from modules.model import FOMOnet
from modules.transcripts import Transcripts
from modules.utils import *

In [2]:
#create runs directory for curves visualization with tensorboard
#shutil.rmtree(f'runs/', ignore_errors=True)
#os.mkdir('runs')
writer = SummaryWriter()
test_writer = SummaryWriter()

In [3]:
#load train & test data
X_train, y_train = pickle.load(open('data/train_split.pkl', 'rb'))
X_test, y_test = pickle.load(open('data/test_split.pkl', 'rb'))

In [4]:
X_train, y_train = X_train[:5], y_train[:5]
X_test, y_test = X_test[:5], y_test[:5]

In [5]:
#pre-processing data for pytorch DataLoader
train_set = Transcripts(X_train, y_train)
test_set = Transcripts(X_test, y_test)

In [6]:
#hyperparameters
batch_size = 2
epochs = 30
lr = 0.0005

In [7]:
#create DataLoader object for train & test data
train_loader = DataLoader(train_set, batch_size=batch_size, collate_fn=pack_seqs, shuffle=True, num_workers=24)
test_loader = DataLoader(test_set, batch_size=batch_size, collate_fn=pack_seqs, shuffle=True, num_workers=24)

/home/xavier/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
#instantiate model, optimizer and loss function
fomonet = FOMOnet(num_channels=4)
optimizer = optim.Adam(fomonet.parameters(), lr)
loss_function = nn.BCELoss(reduction='none')

In [47]:
X_train[0]

tensor([2., 4., 4.,  ..., 1., 1., 1.])

In [49]:
one_hot(X_train[:1])

tensor([[[0., 0., 0.,  ..., 1., 1., 1.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 1.,  ..., 0., 0., 0.]]])

In [9]:
#train model
best_model = 1.0
for epoch in range(epochs):
    fomonet.train()
    for batch in train_loader:
        X = batch[0].view(len(batch[0]),1,-1)
        y = batch[1].view(len(batch[1]),-1)
        X_one_hot = batch[2].view(len(batch[0]),4,-1)
        outputs = fomonet(X_one_hot).view(len(batch[0]),-1)
        fomonet.zero_grad()
        loss = get_loss(outputs, X, y, loss_function)
        loss.backward()
        optimizer.step()
        writer.add_scalar("Loss/train", loss, epoch)
    fomonet.eval()
    test_losses = []
    for batch in test_loader:
        X = batch[0].view(len(batch[0]),1,-1)
        y = batch[1].view(len(batch[1]),-1)
        X_one_hot = batch[2].view(len(batch[0]),4,-1)
        outputs = fomonet(X_one_hot).view(len(batch[0]),-1)
        test_loss = get_loss(outputs, X, y, loss_function)
        test_loss = test_loss.cpu().detach().numpy()
        test_losses.append(test_loss)
        test_writer.add_scalar("Loss/test", test_loss, epoch)
    print(f'{epoch}_{np.mean(test_losses)}')
    if np.mean(test_losses) < best_model:
        best_model = np.mean(test_losses)
        torch.save(fomonet.state_dict(), 'trained_models/fomonet.pt')

0_0.6961261630058289
1_0.6956788897514343


KeyboardInterrupt: 

In [ ]:
writer.flush()
test_writer.flush()
writer.close()
test_writer.close()